# Facility Loaction

## The p-Median Problem

### Problem Definition

The p-Median problem is a classical facility location problem in operations research. The objective is to locate a fixed number of facilities to minimize the total weighted distance between customers and their nearest facilities.

### Parameters:
- $\mathcal{I} = \{1, 2, \ldots, n\}$: set of customers
- $\mathcal{J} = \{1, 2, \ldots, m\}$: set of candidate facility sites  
- $c_{ij}$: distance/cost to serve customer $i \in \mathcal{I}$ by facility $j \in \mathcal{J}$
- $p$: number of facilities to open

### Decision Variables:
- $x_j \in \{0,1\}$: equals 1 if and only if facility $j \in \mathcal{J}$ will be open
- $y_{ij} \in \{0,1\}$: equals 1 if and only if customer $i \in \mathcal{I}$ is assigned to facility $j \in \mathcal{J}$ to be served

### Objective:
Determine $p$ facilities to open and assign customers to facilities to minimize the total service distance/cost.

### Mathematical Formulation

$$
\begin{align}
\text{Minimize total distance/cost} \quad & \sum_{i \in \mathcal{I}} \sum_{j \in \mathcal{J}} c_{ij} y_{ij} \\
\text{Subject to:} \quad & \sum_{j \in \mathcal{J}} y_{ij} = 1, \quad \forall i \in \mathcal{I} \\
& \sum_{j \in \mathcal{J}} x_j = p \\
& y_{ij} \leq x_j, \quad \forall i \in \mathcal{I}, j \in \mathcal{J} \\
& x_j \in \{0,1\}, \quad \forall j \in \mathcal{J} \\
& y_{ij} \in \{0,1\}, \quad \forall i \in \mathcal{I}, j \in \mathcal{J}
\end{align}
$$

### Explanation of Constraints:
1. Each customer must be assigned to exactly one facility
2. Exactly $p$ facilities must be opened
3. A customer can only be assigned to an open facility
4. Binary constraints on decision variables



## The p-Center Problem

### Problem Definition

The p-Center problem is another classical facility location problem in operations research. Unlike the p-Median problem which minimizes the total distance, the p-Center problem focuses on minimizing the maximum distance between any customer and its assigned facility. This model prioritizes fairness by ensuring that the worst-case service distance is as small as possible.

### Parameters:
- $\mathcal{I} = \{1, 2, \ldots, n\}$: set of customers
- $\mathcal{J} = \{1, 2, \ldots, m\}$: set of candidate facility sites
- $c_{ij}$: distance/cost to serve customer $i \in \mathcal{I}$ by facility $j \in \mathcal{J}$
- $p$: number of facilities to open

### Decision Variables:
- $x_j \in \{0,1\}$: equals 1 if and only if facility $j \in \mathcal{J}$ will be open
- $y_{ij} \in \{0,1\}$: equals 1 if and only if customer $i \in \mathcal{I}$ is assigned to facility $j \in \mathcal{J}$ to be served
- $z$: maximum distance from any customer to its assigned facility

### Objective:
Determine $p$ facilities to open and assign customers to facilities to minimize the maximum service distance.

### Mathematical Formulation

The p-Center problem can be formulated as:

$$
\begin{align}
\text{Minimize} \quad & z \\
\text{Subject to:} \quad & \sum_{j \in \mathcal{J}} y_{ij} = 1, \quad \forall i \in \mathcal{I} \\
& \sum_{j \in \mathcal{J}} x_j = p \\
& y_{ij} \leq x_j, \quad \forall i \in \mathcal{I}, j \in \mathcal{J} \\
& \sum_{j \in \mathcal{J}} c_{ij} y_{ij} \leq z, \quad \forall i \in \mathcal{I} \\
& x_j \in \{0,1\}, \quad \forall j \in \mathcal{J} \\
& y_{ij} \in \{0,1\}, \quad \forall i \in \mathcal{I}, j \in \mathcal{J}
\end{align}
$$

### Explanation of Constraints:
1. Each customer must be assigned to exactly one facility
2. Exactly $p$ facilities must be opened
3. A customer can only be assigned to an open facility
4. The variable $z$ represents the maximum distance from any customer to its assigned facility
5. Binary constraints on decision variables



In [2]:
import numpy as np
import pandas as pd
import gurobipy as gp
import folium
from IPython.display import display

# ---- data ----
# read data for all cities
data = pd.read_csv("cn.csv")

# only capitals
data = data[(data["capital"] == "admin") | (data["capital"] == "primary")]

# parameters
cities = data["city"].values
loc_x_vals = data["lat"].values
loc_y_vals = data["lng"].values
loc_x = dict(zip(cities, loc_x_vals))
loc_y = dict(zip(cities, loc_y_vals))

I = cities
J = cities
n = data.shape[0]  # number of cities
p = round(n * 0.2)  # number of facilities to open

# calculate distance
c = {
    (i, j): ((loc_x[i] - loc_x[j]) ** 2 + (loc_y[i] - loc_y[j]) ** 2) ** 0.5
    for i in I
    for j in J
}

# model
m = gp.Model("pmedian")

# decision variable
x = m.addVars(J, vtype=gp.GRB.BINARY, name="x")  # x[j]: if facility j is opened
y = m.addVars(
    I, J, vtype=gp.GRB.BINARY, name="y"
)  # y[i,j]: if customer i is served by facility j

# objective
m.setObjective(y.prod(c), gp.GRB.MINIMIZE)
# m.setObjective(gp.quicksum(c[i, j] * y[i, j] for i in I for j in J), gp.GRB.MINIMIZE)  # alternative way to set objective

# constraints
m.addConstrs(
    (gp.quicksum(y[i, j] for j in J) == 1 for i in I), name="cover"
)  # each customer must be served
m.addConstr(gp.quicksum(x[j] for j in J) == p, name="p_location")  # open p facilities
m.addConstrs(
    (y[i, j] <= x[j] for i in I for j in J), name="capacity"
)  # can only serve from open facilities

# optimize
m.params.timelimit = 30
m.optimize()

# print
print("The solution time is", m.runtime)
print("The MIP gap is", m.mipgap)

# analysis
if m.status != gp.GRB.INFEASIBLE:
    pmedian_map = folium.Map(location=[34.32, 108.55], zoom_start=4)  # center of China

    # mark the capitals
    cities_selected = [j for j in I if x[j].x > 0.9]  # open facilities
    for i in I:
        if i in cities_selected:
            folium.Marker(
                location=(loc_x[i], loc_y[i]), icon=folium.Icon(color="red"), popup=i
            ).add_to(pmedian_map)
        else:
            folium.Marker(location=(loc_x[i], loc_y[i]), popup=i).add_to(pmedian_map)

    # plot assignment decisions
    assignments = [(i, j) for i in I for j in J if y[i, j].x > 0.9]
    # plot results
    for i, j in assignments:
        folium.PolyLine([(loc_x[i], loc_y[i]), (loc_x[j], loc_y[j])]).add_to(
            pmedian_map
        )


display(pmedian_map)  # display map in Jupyter Notebook
pmedian_map.save("pmedian_map.html")  # save map to HTML file

ModuleNotFoundError: No module named 'gurobipy'

## Fixed-Charge Facility Location Problem

### Parameters
- $\mathcal{I} = \{1, 2, \dots, n\}$: Set of customers  
- $\mathcal{J} = \{1, 2, \dots, m\}$: Set of candidate facility sites  
- $d_i$: Demand of customer $i \in \mathcal{I}$  
- $f_j$: Fixed cost to open facility $j \in \mathcal{J}$  
- $v_j$: Service capacity of facility $j \in \mathcal{J}$  
- $c_{ij}$: Per-unit cost to serve customer $i$ from facility $j$  

### Decision Variables
- $x_j \in \{0, 1\}$: 1 if facility $j$ is opened, 0 otherwise  
- $y_{ij} \in [0, 1]$: Fraction of demand of customer $i$ served by facility $j$  

### Objective
Minimize the total cost (fixed cost + variable cost):
$$
\min \sum_{j \in \mathcal{J}} f_j x_j + \sum_{i \in \mathcal{I}} \sum_{j \in \mathcal{J}} c_{ij} d_i y_{ij}
$$

### Subject to
1. **Demand satisfaction (each customer's demand must be met):**
$$
\sum_{j \in \mathcal{J}} y_{ij} = 1 \quad \forall i \in \mathcal{I}
$$

2. **Facility capacity constraint (cannot exceed capacity if opened):**
$$
\sum_{i \in \mathcal{I}} d_i y_{ij} \leq v_j x_j \quad \forall j \in \mathcal{J}
$$

3. **Domain constraints:**
$$
x_j \in \{0,1\} \quad \forall j \in \mathcal{J}
$$
$$
y_{ij} \in [0,1] \quad \forall i \in \mathcal{I}, j \in \mathcal{J}
$$

### Notes
- The model balances trade-offs between fixed costs of opening facilities and variable costs of servicing customers.
- Fractional assignments ($y_{ij}$) are allowed, meaning demand can be split across facilities if feasible.


In [5]:
import numpy as np
import pandas as pd
import gurobipy as gp
import folium
from IPython.display import display

# ----- data ------
# read data for all cities
data = pd.read_csv("cn.csv")
# only captials
data = data[(data["capital"] == "admin") | (data["capital"] == "primary")]

# parameters
n = data.shape[0]
loc_x_vals = data["lat"].values
loc_y_vals = data["lng"].values
pop_vals = data["population"].values
cities = data["city"].values
loc_x = dict(zip(cities, loc_x_vals))
loc_y = dict(zip(cities, loc_y_vals))

I = cities
J = cities
d = dict(zip(cities, pop_vals * 1e-5))
f = {j: 1e3 for j in J}
v = {j: 3e2 for j in J}

# calculate distance
c = {
    (i, j): ((loc_x[i] - loc_x[j]) ** 2 + (loc_y[i] - loc_y[j]) ** 2) ** 0.5
    for i in I
    for j in J
}

# ----- model ------
m = gp.Model("fixed_charge_facility_location")

# decision variables
x = m.addVars(J, vtype=gp.GRB.BINARY, name="x")  # x[j]: if facility j is opened
y = m.addVars(
    I, J, vtype=gp.GRB.CONTINUOUS, name="y"
)  # y[i,j]: fraction of demand of customer i served by facility j

# objective: minimize total cost (fixed cost + variable cost)
m.setObjective(
    gp.quicksum(f[j] * x[j] for j in J)
    + gp.quicksum(c[i, j] * d[i] * y[i, j] for i in I for j in J),
    gp.GRB.MINIMIZE,
)

# constraints
# Demand satisfaction: each customer's demand must be fully met
m.addConstrs(
    (gp.quicksum(y[i, j] for j in J) == 1 for i in I), name="demand_satisfaction"
)

# Facility capacity: cannot exceed capacity if the facility is opened
m.addConstrs(
    (gp.quicksum(d[i] * y[i, j] for i in I) <= v[j] * x[j] for j in J), name="capacity"
)

# Domain constraints: y[i, j] can only be positive if x[j] = 1
m.addConstrs(
    (y[i, j] <= x[j] for i in I for j in J), name="assignment_to_open_facility"
)

# optimize
m.params.timelimit = 30  # set a time limit for the optimization
m.optimize()

# analysis
if m.status == gp.GRB.OPTIMAL:
    print("Optimal solution found!")
    print(f"Total cost: {m.objVal}")
    open_facilities = [j for j in J if x[j].x > 0.9]
    print(f"Open facilities: {open_facilities}")
    assignments = [(i, j) for i in I for j in J if y[i, j].x > 0.01]
    print(f"Assignments: {assignments}")
else:
    print("No optimal solution found within the time limit.")

facility_map = folium.Map(location=[34.32, 108.55], zoom_start=4)  # center of China

# mark the facilities
open_facilities = [j for j in J if x[j].x > 0.9]  # open facilities
for j in open_facilities:
    folium.Marker(
        location=(loc_x[j], loc_y[j]),
        icon=folium.Icon(color="red", icon="home"),
        popup=f"Facility: {j}",
    ).add_to(facility_map)

# Mark the customers
assignments = [(i, j) for i in I for j in J if y[i, j].x > 0.01]
for i, j in assignments:
    folium.Marker(
        location=(loc_x[i], loc_y[i]),
        icon=folium.Icon(color="blue", icon="user"),
        popup=f"Customer: {i}",
    ).add_to(facility_map)
    # Draw lines between customers and their assigned facilities
    folium.PolyLine(
        [(loc_x[i], loc_y[i]), (loc_x[j], loc_y[j])], color="green", weight=1
    ).add_to(facility_map)

# Display the map in Jupyter Notebook
display(facility_map)

# Save the map to an HTML file
facility_map.save("fixed_charge_facility_location_map.html")
# ---- end of code ----

Restricted license - for non-production use only - expires 2026-11-23
Set parameter TimeLimit to value 30
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-1360P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  30

Optimize a model with 1088 rows, 1056 columns and 4128 nonzeros
Model fingerprint: 0x5e0105a9
Variable types: 1024 continuous, 32 integer (32 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [5e+01, 8e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 51240.883486
Presolve time: 0.00s
Presolved: 1088 rows, 1056 columns, 4128 nonzeros
Variable types: 1024 continuous, 32 integer (32 binary)

Root relaxation: objective 1.350717e+04, 280 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    | 